In [ ]:
# Install necessary libraries
!pip install opencv-python-headless
!pip install tensorflow

**Live Emotion Detector**

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# Path to your saved model
model_path = "/content/raf_db_resnet50_best_model.keras"

# Load the model
model = tf.keras.models.load_model(model_path)
print("Model loaded successfully!")

# Define emotion classes
class_names = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

# Preprocessing function for Haar Cascade
def preprocess_face_for_haarcascade(face):
    # Haar Cascade uses grayscale images for detection
    gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

    return gray_face

# Preprocessing function for ResNet model
def preprocess_face_for_resnet(face):
    img_size = (224, 224)  # Model's input size

    # Convert to RGB (ensure 3 channels are in the correct order)
    face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    #face_rgb = face

    # Resize to model's input size
    face_resized = cv2.resize(face_rgb, img_size)

    # Expand dimensions to match model input shape (1, 224, 224, 3)
    face_expanded = np.expand_dims(face_resized, axis=0)
    return face_expanded

# Capture a frame from webcam
def capture_frame():
    js = """
    async function captureFrame() {
        const video = document.createElement('video');
        video.width = 640;
        video.height = 480;

        const stream = await navigator.mediaDevices.getUserMedia({ video: true });
        video.srcObject = stream;
        await new Promise((resolve) => video.onloadedmetadata = resolve);
        video.play();

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        const context = canvas.getContext('2d');
        context.drawImage(video, 0, 0, canvas.width, canvas.height);

        stream.getTracks().forEach(track => track.stop());
        return canvas.toDataURL('image/jpeg');
    }
    captureFrame();
    """
    data = eval_js(js)
    binary = b64decode(data.split(",")[1])
    return np.frombuffer(binary, dtype=np.uint8)

# Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Real-time emotion detection loop
try:
    while True:
        # Capture frame
        print("Capturing frame from webcam...")
        frame_data = capture_frame()
        img = cv2.imdecode(frame_data, cv2.IMREAD_COLOR)

        # Preprocess image for Haar Cascade (Grayscale)
        gray_frame = preprocess_face_for_haarcascade(img)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(
            gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)
        )

        for (x, y, w, h) in faces:
            # Extract face ROI
            face_roi = img[y:y+h, x:x+w]

            # Preprocess the face for ResNet model
            processed_face = preprocess_face_for_resnet(face_roi)

            # Predict emotion
            predictions = model.predict(processed_face, verbose=0)
            emotion_idx = np.argmax(predictions)
            emotion_confidence = predictions[0][emotion_idx] * 100  # Confidence as a percentage
            emotion = class_names[emotion_idx]

            # Debugging: Print raw prediction probabilities
            print(f"Predictions: {predictions[0]}")
            print(f"Detected Emotion: {emotion} ({emotion_confidence:.2f}%)")

            # Draw rectangle and label on the frame
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img, f"{emotion} ({emotion_confidence:.1f}%)",
                        (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Display the frame with detections
        print("Frame with Detections:")
        cv2_imshow(img)

except KeyboardInterrupt:
    print("Live emotion detection stopped.")


**Input Based Emotion Detector**

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# Path to your saved model
model_path = "/content/raf_db_resnet50_best_model.keras"

# Load the model
model = tf.keras.models.load_model(model_path)
print("Model loaded successfully!")

# Define emotion classes
class_names = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

# Preprocessing function for Haar Cascade
def preprocess_face_for_haarcascade(face):
    # Haar Cascade uses grayscale images for detection
    gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    return gray_face

# Preprocessing function for ResNet model (No Normalization)
def preprocess_face_for_resnet(face):
    img_size = (224, 224)  # Model's input size

    # Convert to RGB (ensure 3 channels are in the correct order)
    face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

    # Resize to model's input size
    face_resized = cv2.resize(face_rgb, img_size)

    # Expand dimensions to match model input shape (1, 224, 224, 3)
    face_expanded = np.expand_dims(face_resized, axis=0)
    return face_expanded

# Capture a frame from webcam
def capture_frame():
    js = """
    async function captureFrame() {
        const video = document.createElement('video');
        video.width = 640;
        video.height = 480;

        const stream = await navigator.mediaDevices.getUserMedia({ video: true });
        video.srcObject = stream;
        await new Promise((resolve) => video.onloadedmetadata = resolve);
        video.play();

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        const context = canvas.getContext('2d');
        context.drawImage(video, 0, 0, canvas.width, canvas.height);

        stream.getTracks().forEach(track => track.stop());
        return canvas.toDataURL('image/jpeg');
    }
    captureFrame();
    """
    data = eval_js(js)
    binary = b64decode(data.split(",")[1])
    return np.frombuffer(binary, dtype=np.uint8)

# Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Function to process and classify a single picture
def process_and_classify_image():
    print("Press Enter to capture a picture...")
    input()  # Wait for user input

    # Capture frame
    print("Capturing frame...")
    frame_data = capture_frame()
    img = cv2.imdecode(frame_data, cv2.IMREAD_COLOR)

    # Preprocess image for Haar Cascade (Grayscale)
    gray_frame = preprocess_face_for_haarcascade(img)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(
        gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)
    )

    if len(faces) == 0:
        print("No faces detected.")
    else:
        for (x, y, w, h) in faces:
            # Extract face ROI
            face_roi = img[y:y+h, x:x+w]

            # Preprocess the face for ResNet model
            processed_face = preprocess_face_for_resnet(face_roi)

            # Predict emotion
            predictions = model.predict(processed_face, verbose=0)
            emotion_idx = np.argmax(predictions)
            emotion_confidence = predictions[0][emotion_idx] * 100  # Confidence as a percentage
            emotion = class_names[emotion_idx]

            # Debugging: Print raw prediction probabilities
            print(f"Predictions: {predictions[0]}")
            print(f"Detected Emotion: {emotion} ({emotion_confidence:.2f}%)")

            # Draw rectangle and label on the frame
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img, f"{emotion} ({emotion_confidence:.1f}%)",
                        (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame with detections
    print("Frame with Detections:")
    cv2_imshow(img)

# Main loop for capturing and classifying images
try:
    while True:
        process_and_classify_image()
except KeyboardInterrupt:
    print("Emotion detection stopped.")
